# Importing the dataset

In [15]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
import re
import random 
import numpy as np
import warnings
import tensorflow as tf 
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from preprocess_days_stats import preprocess_match_days
from preprocess_time_serie import preprocess_teams, create_time_series_features
from preprocess_time_series_features import preprocess_features_time_series, create_fast_preprocessing_ts
from tensorflow.keras import layers
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams, plot_loss_curve

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

df_giornate = preprocess_match_days(r"c:\Users\Hp\Documents\Serie_A_dump\csv_serie_a")

Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (18).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!


In [16]:
df_giornate.head()

,div,giornata,stagione,date,hometeam,awayteam,ft_home_goals,ft_away_goals,ft_result,ht_home_goals,ht_away_goals,ht_results,home_shots,away_shots,home_shots_targ,away_shots_targ,home_fouls,away_fouls,home_corners,away_corners,home_yellow,away_yellow,home_red,away_red,home_win_odds,draw_odds,away_win_odds
0,I1,1,2005/2006,2005-08-27,Fiorentina,Sampdoria,2.0,1.0,H,2.0,0.0,H,15.0,9.0,9.0,4.0,18.0,23.0,9.0,3.0,1.0,4.0,0.0,0.0,2.20,2.87,3.25
1,I1,1,2005/2006,2005-08-27,Livorno,Lecce,2.0,1.0,H,1.0,1.0,D,17.0,6.0,7.0,5.0,27.0,21.0,9.0,2.0,2.0,3.0,0.0,0.0,1.90,2.90,4.00
2,I1,1,2005/2006,2005-08-28,Ascoli,Milan,1.0,1.0,D,0.0,0.0,D,8.0,16.0,3.0,9.0,22.0,16.0,0.0,5.0,2.0,1.0,0.0,0.0,7.50,3.75,1.40
3,I1,1,2005/2006,2005-08-28,Inter,Treviso,3.0,0.0,H,1.0,0.0,H,16.0,7.0,9.0,3.0,13.0,20.0,6.0,1.0,1.0,3.0,0.0,0.0,1.16,5.50,13.00
4,I1,1,2005/2006,2005-08-28,Juventus,Chievo,1.0,0.0,H,1.0,0.0,H,16.0,2.0,7.0,0.0,16.0,12.0,6.0,2.0,1.0,2.0,0.0,0.0,1.25,5.00,13.00


# Data preprocessing

In [17]:
all_features = ['ft_goals','ft_goals_conceded','shots','shots_target', 'fouls_done','corners_obtained', 'yellows', 'reds']
less_features = ['ft_goals','ft_goals_conceded','shots', 'fouls_done','corners_obtained', 'reds']
few_features = ['ft_goals','ft_goals_conceded','shots', 'reds']

In [18]:
num_features = 'all'
Statistiche_squadre_dict = preprocess_teams(dataframe = df_giornate)
df_Serie_A = create_time_series_features(num_features, Statistiche_squadre_dict, df_giornate, 5).dropna()
df_Serie_A[(df_Serie_A['hometeam']=='Lecce') | (df_Serie_A['awayteam']=='Lecce')].head(6)

utilizzando tutte le features
preprocess finished


,div,giornata,stagione,date,hometeam,awayteam,home_ft_goals_1,home_ft_goals_2,home_ft_goals_3,home_ft_goals_4,home_ft_goals_5,home_ft_goals_conceded_1,home_ft_goals_conceded_2,home_ft_goals_conceded_3,home_ft_goals_conceded_4,home_ft_goals_conceded_5,home_shots_1,home_shots_2,home_shots_3,home_shots_4,home_shots_5,home_shots_target_1,home_shots_target_2,home_shots_target_3,home_shots_target_4,home_shots_target_5,home_fouls_done_1,home_fouls_done_2,home_fouls_done_3,home_fouls_done_4,home_fouls_done_5,home_corners_obtained_1,home_corners_obtained_2,home_corners_obtained_3,home_corners_obtained_4,home_corners_obtained_5,home_yellows_1,home_yellows_2,home_yellows_3,home_yellows_4,home_yellows_5,home_reds_1,home_reds_2,home_reds_3,home_reds_4,home_reds_5,away_ft_goals_1,away_ft_goals_2,away_ft_goals_3,away_ft_goals_4,away_ft_goals_5,away_ft_goals_conceded_1,away_ft_goals_conceded_2,away_ft_goals_conceded_3,away_ft_goals_conceded_4,away_ft_goals_conceded_5,away_shots_1,away_shots_2,away_shots_3,away_shots_4,away_shots_5,away_shots_target_1,away_shots_target_2,away_shots_target_3,away_shots_target_4,away_shots_target_5,away_fouls_done_1,away_fouls_done_2,away_fouls_done_3,away_fouls_done_4,away_fouls_done_5,away_corners_obtained_1,away_corners_obtained_2,away_corners_obtained_3,away_corners_obtained_4,away_corners_obtained_5,away_yellows_1,away_yellows_2,away_yellows_3,away_yellows_4,away_yellows_5,away_reds_1,away_reds_2,away_reds_3,away_reds_4,away_reds_5,ft_result,home_win_odds,draw_odds,away_win_odds
55,I1,6,2005/2006,2005-10-02,Lecce,Cagliari,0.0,1.0,0.0,0.0,1.0,1.0,3.0,3.0,0.0,2.0,12.0,10.0,7.0,9.0,6.0,2.0,4.0,5.0,6.0,5.0,24.0,18.0,19.0,18.0,21.0,5.0,2.0,3.0,9.0,2.0,4.0,3.0,1.0,2.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,3.0,1.0,1.0,2.0,8.0,9.0,16.0,9.0,9.0,3.0,5.0,8.0,3.0,6.0,12.0,16.0,20.0,12.0,19.0,2.0,5.0,8.0,6.0,4.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,H,2.10,3.00,3.75
69,I1,7,2005/2006,2005-10-16,Reggina,Lecce,1.0,2.0,0.0,1.0,2.0,2.0,0.0,1.0,3.0,3.0,8.0,6.0,8.0,16.0,11.0,1.0,4.0,3.0,8.0,7.0,10.0,20.0,19.0,11.0,24.0,1.0,4.0,5.0,5.0,5.0,0.0,4.0,4.0,3.0,3.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,3.0,0.0,11.0,12.0,10.0,7.0,9.0,8.0,2.0,4.0,5.0,6.0,24.0,24.0,18.0,19.0,18.0,4.0,5.0,2.0,3.0,9.0,0.0,4.0,3.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,H,2.10,2.80,3.50
73,I1,8,2005/2006,2005-10-23,Lecce,Juventus,0.0,3.0,0.0,1.0,0.0,2.0,0.0,1.0,3.0,3.0,9.0,11.0,12.0,10.0,7.0,4.0,8.0,2.0,4.0,5.0,25.0,24.0,24.0,18.0,19.0,3.0,4.0,5.0,2.0,3.0,4.0,0.0,4.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,14.0,8.0,8.0,7.0,17.0,10.0,5.0,6.0,3.0,7.0,24.0,29.0,27.0,18.0,18.0,6.0,1.0,5.0,1.0,8.0,2.0,2.0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,A,7.00,3.75,1.50
86,I1,9,2005/2006,2005-10-26,Palermo,Lecce,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,4.0,0.0,8.0,9.0,21.0,10.0,16.0,4.0,6.0,10.0,4.0,10.0,18.0,26.0,13.0,21.0,17.0,6.0,4.0,11.0,7.0,10.0,3.0,3.0,0.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,3.0,2.0,0.0,1.0,3.0,5.0,9.0,11.0,12.0,10.0,2.0,4.0,8.0,2.0,4.0,30.0,25.0,24.0,24.0,18.0,9.0,3.0,4.0,5.0,2.0,1.0,4.0,0.0,4.0,3.0,0.0,1.0,0.0,0.0,1.0,H,1.57,3.50,6.50
94,I1,10,2005/2006,2005-10-30,Lecce,Messina,0.0,0.0,0.0,3.0,0.0,3.0,3.0,2.0,0.0,1.0,12.0,5.0,9.0,11.0,12.0,5.0,2.0,4.0,8.0,2.0,16.0,30.0,25.0,24.0,24.0,7.0,9.0,3.0,4.0,5.0,3.0,1.0,4.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,4.0,4.0,14.0,10.0,8.0,18.0,6.0,8.0,3.0,4.0,10.0,3.0,22.0,16.0,26.0,15.0,16.0,3.0,3.0,1.0,5.0,2.0,4.0,3.0,4.0,4.0,2.0,0.0,1.0,0.0,0.0,0.0,A,2.00,3.00,3.50
108,I1,11,2005/2006,2005-11-06,Parma,Lecce,0.0,1.0,1.0,1.0,1.0,2.0,1.0,4.0,1.0,3.0,8.0,11.0,9.0,5.0,10.0,5.0,5.0,4.0,2.0,4.0,13.0,15.0,17.0,16.0,22.0,5.0,4.0,5.0,1.0,5.0,1.0,1.0,2.0,3.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,2.0,3.0,3.0,2.0,0.0,14.0,12.0,5.0,9.0,11.0,6.0,5.0,2.0,4.0,8.0,22.0,16.0,30.0,25.0,24.0,5.0,7.0,9.0,3.0,4.0,2.0,3.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,H,1.80,3.25,4.00


In [19]:
(Train_teams_encoded, Valid_teams_encoded, Test_teams_encoded, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
    Train_dict_features_norm, Valid_dict_features_norm, Test_dict_features_norm, 
    Train_teams, Valid_teams, Test_teams, Train_labels, Valid_labels, Test_labels, Train_dict_features, Valid_dict_features, Test_dict_features, 
    Train_odds, Valid_odds, Test_odds) = preprocess_features_time_series(df_Serie_A, num_features, random_state=True)

utilizzando tutte le features
utilizzando tutte le features
utilizzando tutte le features


## Controlliamo che le numeriche tornino tutte

### Caso all features

In [20]:
numero_features = len(all_features)*5*2
numero_features, len(Train_labels), len(Valid_labels), len(Test_labels)

(80, 5076, 628, 301)

In [21]:
for i in (Train_teams_encoded, Valid_teams_encoded, Test_teams_encoded, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
    ):
    print(f' shape: {i.shape}')

for i in list(Train_dict_features_norm.keys()):
    print(Train_dict_features_norm[i].shape)

for i in list(Valid_dict_features_norm.keys()):
    print(Valid_dict_features_norm[i].shape)

 shape: (5076, 86)
 shape: (628, 86)
 shape: (301, 86)
 shape: ()
 shape: ()
 shape: ()
(5076, 10)
(5076, 10)
(5076, 10)
(5076, 10)
(5076, 10)
(5076, 10)
(5076, 10)
(5076, 10)
(628, 10)
(628, 10)
(628, 10)
(628, 10)
(628, 10)
(628, 10)
(628, 10)
(628, 10)


## Creting fast preprocessing datasets

In [24]:
Train_labels_encoded

array(<5076x3 sparse matrix of type '<class 'numpy.float64'>'
	with 5076 stored elements in Compressed Sparse Row format>, dtype=object)

In [22]:
Dataset_train_norm, Dataset_valid_norm, Dataset_Test_norm = create_fast_preprocessing_ts(Train_teams_encoded, Train_dict_features_norm, Train_labels_encoded,
                                                                                         Valid_teams_encoded, Valid_dict_features_norm,
                                                                    Valid_labels_encoded,Test_teams_encoded, Test_dict_features_norm,Test_labels_encoded )

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type csr_matrix).

In [ ]:

# #creo i fast preprocessing datasets
# Dataset_train_norm = tf.data.Dataset.from_tensor_slices(Train_dict_features_norm[list(Train_dict_features_norm.keys())[0]])
# for feature in list(Train_dict_features_norm.keys())[1:]:
#     temp_dataset = tf.data.Dataset.from_tensor_slices(Train_dict_features_norm[feature])
#     Dataset_train_norm = tf.data.Dataset.zip((Dataset_train_norm, temp_dataset))
# Train_labels_encoded = tf.data.Dataset.from_tensor_slices(Train_labels_encoded) # make labels
# Dataset_train_norm = tf.data.Dataset.zip((Dataset_train_norm, Train_labels_encoded))

# #creo un array con le features concatenate
# Dataset_Valid_norm = tf.data.Dataset.from_tensor_slices(Valid_dict_features_norm[list(Valid_dict_features_norm.keys())[0]])
# for feature in list(Valid_dict_features_norm.keys())[1:]:
#     temp_dataset = tf.data.Dataset.from_tensor_slices(Valid_dict_features_norm[feature])
#     Dataset_Valid_norm = tf.data.Dataset.zip((Dataset_Valid_norm, temp_dataset))
# Valid_labels_encoded = tf.data.Dataset.from_tensor_slices(Valid_labels_encoded) # make labels
# Dataset_Valid_norm = tf.data.Dataset.zip((Dataset_Valid_norm, Valid_labels_encoded))

# #creo un array con le features concatenate
# Dataset_Test_norm = tf.data.Dataset.from_tensor_slices(Test_dict_features_norm[list(Test_dict_features_norm.keys())[0]])
# for feature in list(Test_dict_features_norm.keys())[1:]:
#     temp_dataset = tf.data.Dataset.from_tensor_slices(Test_dict_features_norm[feature])
#     Dataset_Test_norm = tf.data.Dataset.zip((Dataset_Test_norm, temp_dataset))
# Test_labels_encoded = tf.data.Dataset.from_tensor_slices(Test_labels_encoded) # make labels
# Dataset_Test_norm = tf.data.Dataset.zip((Dataset_Test_norm, Test_labels_encoded))

# Dataset_train_norm = Dataset_train_norm.batch(32).prefetch(tf.data.AUTOTUNE) #Autotune è per dirgli di prefetchare tanti dati quanti può
# Dataset_valid_norm = Dataset_Valid_norm.batch(32).prefetch(tf.data.AUTOTUNE)
# Dataset_Test_norm = Dataset_Test_norm.batch(32).prefetch(tf.data.AUTOTUNE)

# Model_1: Dense model

In [ ]:
feature_input_shape = Test_dict_features_norm[list(Test_dict_features_norm.keys())[0]].shape[1]
Train_teams_shape = Test_teams_encoded.shape[1]

In [ ]:
all_features = ['ft_goals','ft_goals_conceded','shots','shots_target', 'fouls_done','corners_obtained', 'yellows', 'reds']
less_features = ['ft_goals','ft_goals_conceded','shots', 'fouls_done','corners_obtained', 'reds']
few_features = ['ft_goals','ft_goals_conceded','shots', 'reds']

In [ ]:
#Modello per i teams 
inputs = layers.Input(shape=(Train_teams_shape,))
x = layers.Dense(16, activation = 'relu')(inputs)
outputs = layers.Dense(8)(x)
model_teams = tf.keras.Model(inputs,outputs, name = 'model_1_teams')

# modello ft_goals
inputs = layers.Input(shape=(feature_input_shape,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.1)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dropout(0.0)(x) 
model_ft_goals = tf.keras.Model(inputs, outputs, name='model_1_goals')

# modello ft_goals_conceded
inputs = layers.Input(shape=(feature_input_shape,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.2)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dropout(0.0)(x) 
model_ft_goals_conceded = tf.keras.Model(inputs, outputs, name='ft_goals_conceded')

# modello shots
inputs = layers.Input(shape=(feature_input_shape,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.1)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dropout(0.0)(x) 
model_shots = tf.keras.Model(inputs, outputs, name='model_1_shots')

# modello shots_target
inputs = layers.Input(shape=(feature_input_shape,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.1)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dropout(0.0)(x) 
model_shots_target = tf.keras.Model(inputs, outputs, name='model_1_shots_target')

# modello fouls_done
inputs = layers.Input(shape=(feature_input_shape,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.1)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dropout(0.0)(x) 
model_fouls_done = tf.keras.Model(inputs, outputs, name='model_1_fouls_done')

# modello corners_obtained
inputs = layers.Input(shape=(feature_input_shape,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.1)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dropout(0.0)(x) 
model_corners_obtained = tf.keras.Model(inputs, outputs, name='model_1_corners_obtained')

# modello yellows
inputs = layers.Input(shape=(feature_input_shape,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.1)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dropout(0.0)(x) 
model_yellows = tf.keras.Model(inputs, outputs, name='model_1_corners_yellows')

# modello reds
inputs = layers.Input(shape=(feature_input_shape,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.1)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dropout(0.0)(x) 
model_reds = tf.keras.Model(inputs, outputs, name='model_1_corners_reds')

#Unisco i modelli 
model_1_concat_layer = layers.Concatenate(name="feature_concat")([ model_ft_goals.output, model_ft_goals_conceded.output, 
                                                         model_shots.output, model_shots_target.output, model_fouls_done.output, 
                                                         model_corners_obtained.output, model_yellows.output, model_reds.output])
x = layers.Dense(64, activation='relu')(model_1_concat_layer)
x = layers.Dense(32, activation='relu')(x)
output_layer = layers.Dense(3, activation = 'softmax')(x)

#creo il modello  finale
model_1_final =tf.keras.Model(
    inputs=[[ model_ft_goals.input, model_ft_goals_conceded.input, model_shots.input, model_shots_target.input, model_fouls_done.input, 
                model_corners_obtained.input, model_yellows.input, model_reds.input]],
    outputs=output_layer,
    name='model_1_dense_concat'
)


#compilo il modelllo 
model_1_final.compile(
    loss = 'categorical_crossentropy',
    metrics = 'accuracy',
    optimizer = tf.keras.optimizers.Adam()
    )

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

# fitto il modello 
history_3 = model_1_final.fit(
                Dataset_train_norm,
                epochs = 15,
                verbose = 1,
                validation_data = Dataset_valid_norm
                )

Epoch 1/15
160/160 [==============================] - 3s 4ms/step - loss: 1.0527 - accuracy: 0.4688 - val_loss: 1.0316 - val_accuracy: 0.5042
Epoch 2/15
160/160 [==============================] - 0s 2ms/step - loss: 1.0221 - accuracy: 0.4992 - val_loss: 1.0257 - val_accuracy: 0.5028
Epoch 3/15
160/160 [==============================] - 0s 2ms/step - loss: 1.0125 - accuracy: 0.5025 - val_loss: 1.0233 - val_accuracy: 0.5111
Epoch 4/15
160/160 [==============================] - 0s 2ms/step - loss: 1.0052 - accuracy: 0.5096 - val_loss: 1.0209 - val_accuracy: 0.5236
Epoch 5/15
160/160 [==============================] - 0s 2ms/step - loss: 1.0043 - accuracy: 0.5098 - val_loss: 1.0212 - val_accuracy: 0.5125
Epoch 6/15
160/160 [==============================] - 0s 2ms/step - loss: 1.0028 - accuracy: 0.5116 - val_loss: 1.0212 - val_accuracy: 0.5069
Epoch 7/15
160/160 [==============================] - 0s 2ms/step - loss: 0.9986 - accuracy: 0.5176 - val_loss: 1.0231 - val_accuracy: 0.5056
Epoch 